<a href="https://colab.research.google.com/github/noraleilaa/460_FINAL/blob/main/CoffeeShopProblem_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import simpy
import numpy as np
import random

def customer(env, name, counter):
    start_time = env.now
    activity_time = 9
    print(f'{name}: Arrived at time {start_time}')
    with counter.request() as req:
        yield req
        print(f'{name}: Seized gas pump at time {env.now}')
        yield env.timeout(np.random.normal(10, 2))
        print(f'{name}: Released from gas pump at time {env.now}')
    end_time = env.now
    print(f'{name}: Finished at time {end_time}')

    return {'start_time': start_time, 'end_time': end_time, 'activity_time': activity_time}


def customer_generator(env, counter, total_customers):
    customer_id = 1
    while customer_id <= total_customers:
        yield env.timeout(np.random.exponential(5))
        env.process(customer(env, f'Customer {customer_id}', counter))
        customer_id += 1

def shop_simulation(env, total_customers):
    counter = simpy.Resource(env, capacity=2)

    # simulation results
    results = []

    def customer_wrapper():
        nonlocal results
        while True:
            result = yield from customer(env, f'Customer {len(results) + 1}', counter)
            results.append(result)

    env.process(customer_generator(env, counter, total_customers))
    env.process(customer_wrapper())

    # duration of results
    env.run(until=1000)

    return results

    env.process(customer_generator(env, counter, total_customers))
    env.process(customer_wrapper())

    env.run(until=1000)

    shop_data = pd.DataFrame(results)

    return shop_data

env = simpy.Environment()


shop_data = shop_simulation(env, total_customers=100)
shop_data_df = pd.DataFrame(shop_data)

# Calculate waiting time
shop_data_df['waiting_time'] = shop_data_df['end_time'] - shop_data_df['start_time'] - shop_data_df['activity_time']
waiting_time = shop_data_df['waiting_time']

Customer 1: Arrived at time 0
Customer 1: Seized gas pump at time 0
Customer 1: Arrived at time 4.731344786988421
Customer 1: Seized gas pump at time 4.731344786988421
Customer 1: Released from gas pump at time 8.43994043046917
Customer 1: Finished at time 8.43994043046917
Customer 2: Arrived at time 8.43994043046917
Customer 2: Seized gas pump at time 8.43994043046917
Customer 1: Released from gas pump at time 14.37119638726115
Customer 1: Finished at time 14.37119638726115
Customer 2: Released from gas pump at time 19.615604325495262
Customer 2: Finished at time 19.615604325495262
Customer 3: Arrived at time 19.615604325495262
Customer 3: Seized gas pump at time 19.615604325495262
Customer 2: Arrived at time 27.29388667322462
Customer 2: Seized gas pump at time 27.29388667322462
Customer 3: Released from gas pump at time 31.898647169414986
Customer 3: Finished at time 31.898647169414986
Customer 4: Arrived at time 31.898647169414986
Customer 4: Seized gas pump at time 31.898647169414

In [ ]:
shop_data_df.loc[shop_data_df['waiting_time'] < 0, 'waiting_time'] = 0
shop_data_df['Balking'] = np.where(shop_data_df['waiting_time'] < random.uniform(0, 15), 0, 1)

print(shop_data_df)

    start_time    end_time  activity_time  waiting_time  Balking
0     0.000000    8.439940              9      0.000000        0
1     8.439940   19.615604              9      2.175664        0
2    19.615604   31.898647              9      3.283043        0
3    31.898647   43.679874              9      2.781227        0
4    43.679874   60.506145              9      7.826271        1
5    60.506145   70.145805              9      0.639660        0
6    70.145805   79.897982              9      0.752178        0
7    79.897982   91.434842              9      2.536860        0
8    91.434842  127.211175              9     26.776333        1
9   127.211175  174.987532              9     38.776357        1
10  174.987532  240.141453              9     56.153922        1
11  240.141453  306.582654              9     57.441201        1
12  306.582654  375.640891              9     60.058237        1
13  375.640891  430.389664              9     45.748773        1
14  430.389664  494.10601